In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from tqdm import tqdm

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
DEVICE = "cpu"
seed_torch(2023)

cuda:0


In [3]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.filter_ages(ALL_data,18)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')
ALL_data = ECGHandle.correct_label(ALL_data)
ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ECGHandle.filter_diagnose(ALL_data,'起搏')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'房颤')
# ALL_data = ECGHandle.filter_diagnose(ALL_data,'阻滞')
ALL_data = ECGHandle.remove_duplicated(ALL_data)
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()
seed_torch(2023)
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
# all_dataset = ECGHandle.ECG_Dataset(data_root,ALL_data_buffer,preprocess = True)
####################################################################随机选取test
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
test_dataset = ECGHandle.ECG_Dataset(data_root,test_df,preprocess = True)



            orginal   removed diagnose NaN
   nums      200082          199997       
              HTN             NHTN        
   nums       3273           196724       


            orginal      removed ID NaN   
   nums      199997          199995       
              HTN             NHTN        
   nums       3273           196722       


            orginal            QC         
   nums      199995          72845        
              HTN             NHTN        
   nums       1497           71348        


            orginal      filtered ages    
   nums      72845           69819        
              HTN             NHTN        
   nums       1477           68342        


            orginal   filtered department 
   nums      69819           15344        
              HTN             NHTN        
   nums       1477           13867        


     reset num:       10  
  ERR labels num:     27  
            orginal      correct label    
   nums      15344           15

In [4]:
ECGfile_name = '20-2092'
index_ = (test_dataset.infos[test_dataset.infos['ECGFilename']==(ECGfile_name)]).index.values[0]
row_num = (test_dataset.infos).index.get_loc(index_)
ecg_data = test_dataset.datas[row_num]
print(ecg_data)
index = row_num
waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv')

tensor([[-0.0068, -0.0068, -0.0078,  ...,  0.0410,  0.0410,  0.0439],
        [-0.0166, -0.0166, -0.0176,  ...,  0.0098,  0.0107,  0.0127],
        [-0.0098, -0.0098, -0.0098,  ..., -0.0312, -0.0303, -0.0312],
        ...,
        [-0.0332, -0.0322, -0.0332,  ...,  0.0576,  0.0595,  0.0615],
        [-0.0273, -0.0273, -0.0283,  ...,  0.0420,  0.0439,  0.0459],
        [-0.0254, -0.0254, -0.0244,  ...,  0.0283,  0.0293,  0.0312]])


In [5]:
index = 2

In [6]:
Models_path = '/workspace/data/Interpretable_HTN/model/20230211_072050/20230211_072050/parameter_EarlyStoping_0.pt'

In [7]:
# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    # print('backward_hook ',len(grad_out),grad_out[0].shape)
    grad_block.append(grad_out[0].clone().cpu().detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    # print('farward_hook ',output.shape)
    fmap_block.append(output.clone().cpu().detach())

In [8]:
def scaled_layer_cam(x,gamma:float = 2.):
    x_max = x.max()
    x_scaled = np.tanh(((gamma*x)/x_max))
    return x_scaled

In [9]:
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
x_index = np.arange(0,12)
fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
for index in range(test_dataset.__len__()):
    Models_path = '/workspace/data/Interpretable_HTN/model/20230212_134040/20230212_134040/parameter_EarlyStoping_0.pt'
    save_root = '/workspace/data/Interpretable_HTN/model/20230212_134040/20230212_134040/parameter_EarlyStoping_0/'
    NET = [Net.MLBFNet_GUR(True,True,True,2,Dropout_rate=0.3) ] # type: ignore
    testmodel = NET[0].to(DEVICE)
    testmodel.load_state_dict(torch.load(Models_path))


    testmodel.conv1.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv1.register_full_backward_hook(backward_hook)#反向传播
    testmodel.conv2.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv2.register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[0].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[0].register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[1].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[1].register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[2].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[2].register_full_backward_hook(backward_hook)#反向传播


    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]

    testmodel.eval()
    fmap_block = list()
    grad_block = list()

    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  

    outputs = testmodel(inputs)
    # print('outputs',outputs)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    possibility = nn.functional.softmax(outputs,dim=-1)
    print("labels: {}".format(labels))
    print("predict: {}".format(pred))
    print("possibility: {}".format(possibility))
    loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用

    fmap1 = (fmap_block[0][0]).to('cpu').detach().numpy()
    gradmap1 = (grad_block[4][0]).to('cpu').detach().numpy()#G是从后往前的，conv1的在后面
    layer2d_vlue1 = cam.caculate_layer_cam_vlue_2d(fmap1,gradmap1)

    fmap2 = (fmap_block[1][0]).to('cpu').detach().numpy()
    gradmap2 = (grad_block[3][0]).to('cpu').detach().numpy()
    layer2d_vlue2 = cam.caculate_layer_cam_vlue_2d(fmap2,gradmap2)

    fmap2d_1 = (fmap_block[2][0]).to('cpu').detach().numpy()
    gradmap2d_1 = (grad_block[2][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_1 = cam.caculate_layer_cam_vlue_2d(fmap2d_1,gradmap2d_1)

    fmap2d_2 = (fmap_block[3][0]).to('cpu').detach().numpy()
    gradmap2d_2 = (grad_block[1][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_2 = cam.caculate_layer_cam_vlue_2d(fmap2d_2,gradmap2d_2)

    fmap2d_3 = (fmap_block[4][0]).to('cpu').detach().numpy()
    gradmap2d_3 = (grad_block[0][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_3 = cam.caculate_layer_cam_vlue_2d(fmap2d_3,gradmap2d_3)

    layer_vlue_mean = (scaled_layer_cam(layer2d_vlue1)+scaled_layer_cam(layer2d_vlue2)+scaled_layer_cam(layer2d_vlue2d_1)+scaled_layer_cam(layer2d_vlue2d_2)+scaled_layer_cam(layer2d_vlue2d_3))/5


    cmap = 'OrRd' 
    colors = ECGplot.color_map(layer_vlue_mean, cmap)#相当于归一化

    for i,ax in enumerate(axs.flat):  # type: ignore
        plot_y = np.array(inputs[i]*5000.)#缩放
        t = np.arange(0,5000)
        ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
        for j in t:
            ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value
    plt.savefig(save_root+'/'+info['ECGFilename']+'_'+str(labels[0].tolist())+'_'+str(pred[0].tolist())+'.png', bbox_inches='tight',dpi = 200)  
    plt.close()  

labels: tensor([1])
predict: tensor([1])
possibility: tensor([[0.0019, 0.9981]], grad_fn=<SoftmaxBackward0>)
labels: tensor([1])
predict: tensor([1])
possibility: tensor([[0.4974, 0.5026]], grad_fn=<SoftmaxBackward0>)
labels: tensor([0])
predict: tensor([0])
possibility: tensor([[0.8812, 0.1188]], grad_fn=<SoftmaxBackward0>)
labels: tensor([1])
predict: tensor([1])
possibility: tensor([[0.4481, 0.5519]], grad_fn=<SoftmaxBackward0>)
labels: tensor([1])
predict: tensor([0])
possibility: tensor([[0.9159, 0.0841]], grad_fn=<SoftmaxBackward0>)
labels: tensor([0])
predict: tensor([0])
possibility: tensor([[0.6774, 0.3226]], grad_fn=<SoftmaxBackward0>)
